## It contains the code for extracting tiffs from google earth engine

In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AX4XfWjiMMVDpp91j6Tnbu0h1bYAOsjLFAX7iAC5KaExqjfJthddfckrDf0

Successfully saved authorization token.


In [ ]:
# from google.colab import auth

import numpy as np
import folium as folium
import time

# auth.authenticate_user()
project_id = 'moja-328209'
!gcloud config set project {project_id}
!gsutil ls
%pwd

Updated property [core/project].
gs://data-myanmar-new/
gs://data-philippines-new/
gs://data-uganda/
gs://philippines-backup-new/


'/data/uganda/budongo'

In [ ]:
# Coordinates hard-coded 
# Present in (longitude,latitude) format
# Mabira Forest Reserve
# 32.8546143, 0.5733394
# 32.8635406, 0.4106105
# 33.0228424, 0.3584266
# 33.1395721, 0.4264029
# 32.9768372, 0.6365073
# 32.8546143, 0.5726528
# 32.8546143, 0.5733394
# Bwindi 
# 29.5992279, -0.9653779
# 29.5985413, -0.9667510
# 29.5676422, -1.0100032
# 29.6974182, -1.1514270
# 29.8519135, -1.1006251
# 29.8093414, -0.9440948
# 29.6871185, -0.8479762
# 29.5992279, -0.9653779
# Budongo
# 31.3824463, 1.8014609
# 31.3000488, 1.5461380
# 32.0690918, 1.8179322
# 32.3547363, 2.1473243
# 32.1185303, 2.3696245
# 31.3824463, 1.8014609
#previous area was very small so we are taking new area for Budongo
# 31.3000488, 1.5461380
# 33.6000488, 1.5461380
# 31.3000488, 3.8461380
# 33.6000488, 3.8461380
regions=["mabira","bwindi","budongo"]

In [ ]:
geometry_mabira = ee.Geometry.Polygon(32.8546143, 0.5733394
                              ,32.8635406, 0.4106105
                              ,33.0228424, 0.3584266
                              ,33.1395721, 0.4264029
                              ,32.9768372, 0.6365073
                              ,32.8546143, 0.5726528
                              ,32.8546143, 0.5733394)
geometry_bwindi = ee.Geometry.Polygon(29.5992279, -0.9653779
                              ,29.5985413, -0.9667510
                              ,29.5676422, -1.0100032
                              ,29.6974182, -1.1514270
                              ,29.8519135, -1.1006251
                              ,29.8093414, -0.9440948
                              ,29.6871185, -0.8479762
                              ,29.5992279, -0.9653779)
geometry_budongo = ee.Geometry.Polygon(31.3000488, 1.5461380
                              ,33.6000488, 1.5461380
                              ,33.6000488, 3.8461380
                              ,31.3000488, 3.8461380
                              ,31.3000488, 1.5461380)
mabira_image = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR')\
                .filterBounds(geometry_mabira)\
               .filter(ee.Filter.calendarRange(2000, 2000, 'year'))\
               .filter(ee.Filter.calendarRange(1,12,'month'))\
               .mean()

bwindi_image = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR')\
                .filterBounds(geometry_bwindi)\
               .filter(ee.Filter.calendarRange(2000, 2000, 'year'))\
               .filter(ee.Filter.calendarRange(1,12,'month'))\
               .mean()
budongo_image = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR')\
                .filterBounds(geometry_budongo)\
               .filter(ee.Filter.calendarRange(2000, 2000, 'year'))\
               .filter(ee.Filter.calendarRange(1,12,'month'))\
               .mean()


mabira_image = mabira_image.clip(geometry_mabira)
bwindi_image = bwindi_image.clip(geometry_bwindi)
budongo_image = budongo_image.clip(geometry_budongo)


## Checking if correct section is taken

In [ ]:
# Inspired by code present here https://colab.research.google.com/github/giswqs/qgis-earthengine-examples/blob/master/Folium/ee-api-folium-setup.ipynb#scrollTo=Q7uImKYn2R2z

ndvi_mabira = mabira_image.normalizedDifference(['B4', 'B5'])
ndvi_bwindi = bwindi_image.normalizedDifference(['B4', 'B5'])
ndvi_budongo = budongo_image.normalizedDifference(['B4', 'B5'])

ndvi_viz = {'min': -1.0, 'max': 1.0, 'palette': ['0000FF', '00FF00']}

f = folium.Figure(width=500, height=500)
map_ndvi = folium.Map(location=[1.8014609,31.3824463], zoom_start=7).add_to(f)

# Add the image layer to the map and display it.

folium.raster_layers.TileLayer(
            tiles = ee.Image(ndvi_mabira).getMapId(ndvi_viz)['tile_fetcher'].url_format,
            attr = 'GEE',
            ).add_to(map_ndvi)
folium.raster_layers.TileLayer(
            tiles = ee.Image(ndvi_bwindi).getMapId(ndvi_viz)['tile_fetcher'].url_format,
            attr = 'GEE',
            ).add_to(map_ndvi)
folium.raster_layers.TileLayer(
            tiles = ee.Image(ndvi_budongo).getMapId(ndvi_viz)['tile_fetcher'].url_format,
            attr = 'GEE',
            ).add_to(map_ndvi)

display(map_ndvi)

In [ ]:
# for exporting to cloud storage use the following
task = ee.batch.Export.image.toCloudStorage(**{
        'image': mabira_image,
        'region': geometry_mabira,
        'description':'gearth raw image' + str(2000),
        'fileNamePrefix': "gearth_year_2000_/",
        'scale': 30,
        'bucket': 'data-uganda',
        'fileFormat': 'GeoTIFF',
        'maxPixels': 10000000000000})
task.start()

In [ ]:
import time
sleep_time = 10
for x in range(5):
    print("time: " + str(x * sleep_time) + " seconds")
    if task == None:
        continue
    print(task.status())
    time.sleep(sleep_time)
#  for cancelling any task use - # task[task_no_tmp].cancel()

In [ ]:
#  for downloading selected years
# selected = [2007, 2021]
landsat_pre_1999='LANDSAT/LT05/C01/T1_SR'
landsat_post_1999='LANDSAT/LE07/C01/T1_SR'

# EXPORTING CODE for BEGINS
task = [None] * (150)


In [ ]:
#for budongo
for year in range(1984, 2021):
    my_image = ee.ImageCollection(landsat_pre_1999 if year<1999 else landsat_post_1999)\
                .filterBounds(geometry_budongo)\
               .filter(ee.Filter.calendarRange(year, year, 'year'))\
               .filter(ee.Filter.calendarRange(1,12,'month'))\
               .mean()

    my_image = my_image.clip(geometry_budongo)
    # for exporting to cloud storage use the following
    task_no = (year - 1984)
    task[task_no] = ee.batch.Export.image.toCloudStorage(**{
            'image': my_image,
            'region': geometry_budongo,
            'description':'budongo_landsat7_pp_' + str(year)  + "_" + str(task_no),
            'fileNamePrefix': "budongo/gearth_year" + str(year) + "_/",
            'scale': 30,
            'bucket': 'data-uganda',
            'fileFormat': 'GeoTIFF',
            'maxPixels': 10000000000000})
    task[task_no].start()

print("#### finished starting all the tasks ####")


#### finished starting all the tasks ####


In [ ]:
sleep_time = 120
for x in range(5):
    print("time: " + str(x * sleep_time) + " seconds")
    for year_tmp in range(1984, 2021):
          task_no_tmp = (year_tmp - 1984) 
          if task[task_no_tmp] == None:
              continue
          print(task[task_no_tmp].status())


    time.sleep(sleep_time)

time: 0 seconds
{'state': 'READY', 'description': 'budongo_landsat7_pp_1984_0', 'creation_timestamp_ms': 1637144292905, 'update_timestamp_ms': 1637144292905, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'LEIR2AHHEHGVKSFMAWQYZW7E', 'name': 'projects/earthengine-legacy/operations/LEIR2AHHEHGVKSFMAWQYZW7E'}
{'state': 'READY', 'description': 'budongo_landsat7_pp_1985_1', 'creation_timestamp_ms': 1637144293468, 'update_timestamp_ms': 1637144293468, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'G3ZXHJJTBOR75OE7KKPYJSL3', 'name': 'projects/earthengine-legacy/operations/G3ZXHJJTBOR75OE7KKPYJSL3'}
{'state': 'READY', 'description': 'budongo_landsat7_pp_1986_2', 'creation_timestamp_ms': 1637144293896, 'update_timestamp_ms': 1637144293896, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'AKTYZI7GYOQ4D6Q5MJFQJAKH', 'name': 'projects/earthengine-legacy/operations/AKTYZI7GYOQ4D6Q5MJFQJAKH'}
{'state': 'READY', 'description': 'budongo_landsat7_pp_1987_3', 'crea

In [ ]:
for year_tmp in range(1984, 2021):
          task_no_tmp = (year_tmp - 1984) 
          if task[task_no_tmp] == None:
              continue
          print(task[task_no_tmp].status())

{'state': 'COMPLETED', 'description': 'budongo_landsat7_pp_1984_0', 'creation_timestamp_ms': 1637144292905, 'update_timestamp_ms': 1637144652599, 'start_timestamp_ms': 1637144338851, 'task_type': 'EXPORT_IMAGE', 'destination_uris': ['https://console.developers.google.com/storage/browser/data-uganda/budongo/gearth_year1984_/'], 'attempt': 1, 'id': 'LEIR2AHHEHGVKSFMAWQYZW7E', 'name': 'projects/earthengine-legacy/operations/LEIR2AHHEHGVKSFMAWQYZW7E'}
{'state': 'COMPLETED', 'description': 'budongo_landsat7_pp_1985_1', 'creation_timestamp_ms': 1637144293468, 'update_timestamp_ms': 1637144712484, 'start_timestamp_ms': 1637144343431, 'task_type': 'EXPORT_IMAGE', 'destination_uris': ['https://console.developers.google.com/storage/browser/data-uganda/budongo/gearth_year1985_/'], 'attempt': 1, 'id': 'G3ZXHJJTBOR75OE7KKPYJSL3', 'name': 'projects/earthengine-legacy/operations/G3ZXHJJTBOR75OE7KKPYJSL3'}
{'state': 'COMPLETED', 'description': 'budongo_landsat7_pp_1986_2', 'creation_timestamp_ms': 163

## Downloading matt hansen data


In [ ]:
budongo_image_hansen = ee.Image('UMD/hansen/global_forest_change_2020_v1_8').clip(geometry_budongo)
               
# budongo_image_hansen = budongo_image_hansen.clip(geometry_budongo)
budongo_image_hansen=budongo_image_hansen.select('loss')

# Inspired by code present here https://colab.research.google.com/github/giswqs/qgis-earthengine-examples/blob/master/Folium/ee-api-folium-setup.ipynb#scrollTo=Q7uImKYn2R2z

ndvi_viz = {'min': 0, 'max': 1.0, 'palette': ['00FF00', '0000FF']}

f = folium.Figure(width=500, height=500)
map_ndvi = folium.Map(location=[1.8014609,31.3824463], zoom_start=7).add_to(f)

# Add the image layer to the map and display it.

folium.raster_layers.TileLayer(
            tiles = ee.Image(budongo_image_hansen).getMapId(ndvi_viz)['tile_fetcher'].url_format,
            attr = 'GEE',
            ).add_to(map_ndvi)

display(map_ndvi)

In [ ]:
task = ee.batch.Export.image.toCloudStorage(**{
            'image': ee.Image('UMD/hansen/global_forest_change_2020_v1_8').clip(geometry_budongo),
            'region': geometry_budongo,
            'description':'Matt_hansen_budongo',
            'fileNamePrefix': "budongo/",
            'scale': 30,
            'bucket': 'data-uganda',
            'fileFormat': 'GeoTIFF',
            'maxPixels': 10000000000000})
task.start()

In [ ]:
print(task.status())

{'state': 'FAILED', 'description': 'Matt_hansen_budongo', 'creation_timestamp_ms': 1638274348334, 'update_timestamp_ms': 1638274382531, 'start_timestamp_ms': 1638274367799, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'error_message': 'Export too large: specified 72871830 pixels (max: 51380224). Specify higher maxPixels value if you intend to export a large area.', 'id': 'GZEGFK2L75I4QI336TM6Y7X6', 'name': 'projects/earthengine-legacy/operations/GZEGFK2L75I4QI336TM6Y7X6'}


In [ ]:
#for mabira
for year in range(1984, 2021):
    my_image = ee.ImageCollection(landsat_pre_1999 if year<1999 else landsat_post_1999)\
                .filterBounds(geometry_mabira)\
               .filter(ee.Filter.calendarRange(year, year, 'year'))\
               .filter(ee.Filter.calendarRange(1,12,'month'))\
               .mean()

    my_image = my_image.clip(geometry_mabira)
    # for exporting to cloud storage use the following
    task_no = (year - 1984)
    task[task_no] = ee.batch.Export.image.toCloudStorage(**{
            'image': my_image,
            'region': geometry_mabira,
            'description':'mabira_landsat7_pp_' + str(year)  + "_" + str(task_no),
            'fileNamePrefix': "mabira/gearth_year" + str(year) + "_/",
            'scale': 30,
            'bucket': 'data-uganda',
            'fileFormat': 'GeoTIFF',
            'maxPixels': 10000000000000})
    task[task_no].start()

In [ ]:
#for bwindi
for year in range(1984, 2021):
    my_image = ee.ImageCollection(landsat_pre_1999 if year<1999 else landsat_post_1999)\
                .filterBounds(geometry_bwindi)\
               .filter(ee.Filter.calendarRange(year, year, 'year'))\
               .filter(ee.Filter.calendarRange(1,12,'month'))\
               .mean()

    my_image = my_image.clip(geometry_bwindi)
    # for exporting to cloud storage use the following
    task_no = (year - 1984)
    task[task_no] = ee.batch.Export.image.toCloudStorage(**{
            'image': my_image,
            'region': geometry_bwindi,
            'description':'bwindi_landsat7_pp_' + str(year)  + "_" + str(task_no),
            'fileNamePrefix': "bwindi/gearth_year" + str(year) + "_/",
            'scale': 30,
            'bucket': 'data-uganda',
            'fileFormat': 'GeoTIFF',
            'maxPixels': 10000000000000})
    task[task_no].start()